In [109]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/analysis_lib.ipynb"

In [110]:
args = getargs()

In [111]:
NMERS = 5

In [112]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_3mer.pickle",
#     "ncores": 4,
#     "nmers": 3,
#     "multivariate": True
# }

args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params_{0}mer.pickle".format(NMERS),
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}

In [113]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [114]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = OrderedDict(HMM_PARAMS)
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS.keys() == ALL_KMERS

# Train Model 

In [115]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [116]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [117]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (10/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (39/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [118]:
mk_transmat = mk_transmat2

In [119]:
F = ghmm.Float()  # emission domain of this model

def mk_model_multivariate():
    # example code for a continuous HMM with gaussian emissions

    A = mk_transmat(NMERS)
    B = [ [ 
            [float(df[['mean']].mean()), float(df[['stdv']].mean())], #mu1, mu2
            [x for x in chain(*df[['mean', 'stdv']].cov().values.tolist())], #covariance matrix
        ] for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.MultivariateGaussianDistribution(F), A, B, pi)
    return model

In [120]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = [ [float(df[['mean']].mean()), float(df[['mean']].std())] #mu1, stdv
            for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [121]:
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [122]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=1024)
  state 0 (initial=0.00, mu=67.32, sigma=2.30)
    Transitions: ->0 (0.01), ->1 (0.01), ->2 (0.01), ->3 (0.01), ->4 (0.01), ->5 (0.01), ->6 (0.01), ->7 (0.01), ->8 (0.01), ->9 (0.01), ->10 (0.01), ->11 (0.01), ->12 (0.01), ->13 (0.01), ->14 (0.01), ->15 (0.01)
  state 1 (initial=0.00, mu=64.39, sigma=2.97)
    Transitions: ->1 (0.02), ->4 (0.20), ->5 (0.20), ->6 (0.20), ->7 (0.20), ->16 (0.01), ->17 (0.01), ->18 (0.01), ->19 (0.01), ->20 (0.01), ->21 (0.01), ->22 (0.01), ->23 (0.01), ->24 (0.01), ->25 (0.01), ->26 (0.01), ->27 (0.01), ->28 (0.01), ->29 (0.01), ->30 (0.01), ->31 (0.01)

  ...

  state 1022 (initial=0.00, mu=48.64, sigma=1.80)
    Transitions: ->992 (0.01), ->993 (0.01), ->994 (0.01), ->995 (0.01), ->996 (0.01), ->997 (0.01), ->998 (0.01), ->999 (0.01), ->1000 (0.01), ->1001 (0.01), ->1002 (0.01), ->1003 (0.01), ->1004 (0.01), ->1005 (0.01), ->1006 (0.01), ->1007 (0.01), ->1016 (0.20), ->1017 (0.20), ->1018 (0.20), ->1019 (0.20), ->1022 (0.02)

In [123]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [124]:
def predict(mixed):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    if MULTIVARIATE: 
        emissions = [x for x in chain(*mixed)]    
    else: 
        emissions = [x[0] for x in mixed]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [125]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [126]:
predict(seq)

'GGCTA'

# Validate Model 

In [127]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [128]:
assert os.path.isfile(args["events"])

In [129]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>burn-in lambda_ref']
GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC


In [130]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [131]:
prepare_filemap(file_data)

In [132]:
def train_read(file_obj):    
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [133]:
def basecall_read(file_obj):
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [134]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [ ]:
p = Pool(args["ncores"])

In [ ]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

Prediction: 
done 0.342969%

### Stats

In [ ]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [ ]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [ ]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            fr.write(">ch{0}_file{1}_random".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [ ]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(ref_file, fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

In [ ]:
def mk_stat(t):
    samfile = "{0}.{1}.sorted.bam".format(args["out_basename"], t)
    sst = samstats(samfile, ref, ncores=args["ncores"])
    return pandas.DataFrame(sst.print_summary())

In [ ]:
stats = map(mk_stat, types)
print(types)
side_by_side(*stats)

In [ ]:
for t, df in zip(types, stats):
    with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
        f.write(df.to_html())

In [ ]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [ ]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [ ]:
# consensus

In [ ]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)